In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Review + Reduce Sort


### Agenda 

 - Review Iterate and Reduce
 - Review Generic Devide-and-Conquer
 - 

## Iterate

- Iterate over a sequence and accumulate a result that changes at each step (e.g., "running sum")

$iterate \ (f : \alpha \times \beta \rightarrow \alpha) (x : \alpha) (a : \mathbb{S}_\beta) : \alpha$

$iterate$ is a function that takes as input:
- another function $f : \alpha \times \beta \rightarrow \alpha$
- an initial result $x$
- a sequence $a$ of type $\mathbb{S}_\beta$

and returns a value of type $\alpha$ that is the result of applying $f(x,a)$ to each element of the sequence.


<br>

$iterate \: f \: x \: a =
\begin{cases}
x & \hbox{if} \: |a| = 0\\
iterate \: f \:\: f(x, a[0]) \:\:\: a[1 \ldots |a|-1]& \hbox{otherwise}
\end{cases}
$


e.g.

$iterate \:\: + \:\:\: 0 \:\:\: \langle 2,5,1,6 \rangle \Rightarrow (((2+5)+1)+6) \Rightarrow 14$

<br>

$f(f(f(x, a[0]), a[1]), a[2])\ldots)$

In [3]:
def iterate(f, x, a):
    """
    Params:
      f.....function to apply
      x.....return when a is empty
      a.....input sequence
    """
    print('iterate: calling %s x=%s a=%s' % (f.__name__, x, a))
    if len(a) == 0:
        return x
    else:
        return iterate(f, f(x, a[0]), a[1:])

def plus(x, y):
    return x + y

iterate(plus, 0, [1,2,4,6,8,2,5,1,6])

iterate: calling plus x=0 a=[1, 2, 4, 6, 8, 2, 5, 1, 6]
iterate: calling plus x=1 a=[2, 4, 6, 8, 2, 5, 1, 6]
iterate: calling plus x=3 a=[4, 6, 8, 2, 5, 1, 6]
iterate: calling plus x=7 a=[6, 8, 2, 5, 1, 6]
iterate: calling plus x=13 a=[8, 2, 5, 1, 6]
iterate: calling plus x=21 a=[2, 5, 1, 6]
iterate: calling plus x=23 a=[5, 1, 6]
iterate: calling plus x=28 a=[1, 6]
iterate: calling plus x=29 a=[6]
iterate: calling plus x=35 a=[]


35

Work and Span of iterate with **plus function**?

$W(n) = W(n-1) + 1 \in O(n)$

$S(n) = S(n-1) + 1 \in O(n)$

## Reduce


> A function that repeatedly applies an **associative binary operation** to a collection of elements until the result is *reduced* to a single value.

Associative operations allow commuting the order of operations.
- $plus(plus(2,3), 5) = plus(2, plus(3,5)) = 10$

<br>

**formal definition of reduce**:

$reduce \: (f : \alpha \times \alpha \rightarrow \alpha) (id : \alpha) (a : \mathbb{S}_\alpha) : \alpha$

Input is:
- $f$: an associative binary function
- $a$ is the sequence
- $id$ is the **left identity** of $f$ $\:\: \equiv \:\:$ $f(id, x) = x$ for all $x \in \alpha$

Returns:
- a value of type $\alpha$ that is the result of the "sum" with respect to $f$ of the input sequence $a$


<br>


$reduce \: f \: id \: a =
\begin{cases}
id & \hbox{if} \: |a| = 0\\
a[0] & \hbox{if} \: |a| = 1\\
f(reduce \: f \: id \: (a[0 \ldots \lfloor \frac{|a|}{2} \rfloor - 1]), \\ \:\:\:reduce \: f \: id \: (a[\lfloor \frac{|a|}{2} \rfloor \ldots |a|-1])& \hbox{otherwise}
\end{cases}
$

> When $f$ is associative: $reduce \: f \: id \: a  \: \equiv \: iterate \: f \: id \: a$, reduce is a variant of iterate that allows for easier parallelism.





In [4]:
def reduce(f, id_, a):
    # print('a=%s' % a) # for tracing
    if len(a) == 0:
        return id_
    elif len(a) == 1:
        return a[0]
    else:
        # can call these in parallel
        return f(reduce(f, id_, a[:len(a)//2]), 
                 reduce(f, id_, a[len(a)//2:]))

reduce(plus, 0, [1,2,4,6,8,2,5,1,6])

35

Work and Span of reduce with **plus function**

$$W(n) = 2W(n/2) + 1 \in O(n)$$

$$S(n) = S(n/2) + 1 \in O(\lg n)$$

For more complicated combination functions, we can define a generic version of most divide and conquer algorithms and show that it can be implemented with `reduce` and `map`.

In [ ]:
## Generic divide and conquer algorithm.

def my_divide_n_conquer_alg(mylist):    
    if len(mylist) == 0:
        return LEFT_IDENTITY# <identity>
    elif len(mylist) == 1:
        return BASECASE(mylist[0]) # basecase for 1
    else:
        return COMBINE_FUNCTION(
            my_divide_n_conquer_alg(mylist[:len(mylist)//2]),
            my_divide_n_conquer_alg(mylist[len(mylist)//2:])
        )

def COMBINE_FUNCTION(solution1, solution2):
    """ return the combination of two recursive solutions"""
    pass

def BASECASE(value):
    """ return the basecase value for a single input"""
    pass

### is equivalent to
reduce(COMBINE_FUNCTION, LEFT_IDENTITY, (map(BASECASE, mylist)))

### Example: Sorting with Reduce

In [2]:
def merge(left, right):
    """
    Takes in two sorted lists and returns a sorted list that combines them both.
    """
    i = j = 0
    result = []
    while i < len(left) and j < len(right):
        if right[j] < left[i]:   # out of order: e.g., left=[4], right=[3]
            result.append(right[j])
            j += 1
        else:                   # in order: e.g., left=[1], right=[2]
            result.append(left[i])
            i += 1    
    # append any remaining items (at most one list will have items left)
    result.extend(left[i:])
    result.extend(right[j:])
    return result

merge([1,4,8], [2,3,10])

[1, 2, 3, 4, 8, 10]

What is base case and left identity? 

In [29]:
def singleton(value):
    """ just created a list with one element. """
    return [value]

## reduce(COMBINE_FUNCTION, LEFT_IDENTITY, (map(BASECASE, mylist)))
reduce(merge, [], list(map(singleton, [1,3,6,4,8,7,5,2])))

[1, 2, 3, 4, 5, 6, 7, 8]

What if we use `iterate` instead of `reduce`?

In [30]:
iterate(merge, [], list(map(singleton, [1,3,6,4,8,7,5,2])))

[1, 2, 3, 4, 5, 6, 7, 8]

### Analysis

`iterate(merge, [], list(map(singleton, [1,3,6,4,8,7,5,2])))`

This is **insertion sort**!

- We iterate from left to right.
- At each step we insert the next element into the appropriate place in the sorted list.

$[1] \rightarrow [1,3] \rightarrow [1,3,6] \rightarrow [1,3,4,6] \ldots$

<br><br>

Assuming the `merge` function has **work** $O(n)$.

$$W(n) = W(n-1) + n \in O(n^2)$$

<br><br>

Assuming the `merge` function has **span** $O(\lg n)$ (note our implementation above doesn't yet do this).

$$S(n) = S(n-1) + \lg n \in O(n \lg n)$$


<br><br><br><br>

`reduce(merge, [], list(map(singleton, [1,3,6,4,8,7,5,2])))`

This is **merge sort**!

<br><br>

Assuming the `merge` function has **work** $O(n)$.

$$W(n) = 2W(n/2) + n \in O(n \lg n)$$

Assuming the `merge` function has **span** $O(\lg n)$.

$$S(n) = S(n/2) + \lg n \in O(\lg^2 n)$$

## Scan

*reduce* doesn't store the intermediate results, which limits it somewhat. 

*scan* is like reduce, but it keeps track of the intermediate computations (like *iterate* does).

$scan \: (f : \alpha \times \alpha \rightarrow \alpha) (id : \alpha) (a : \mathbb{S}_\alpha) : (S_\alpha * \alpha)$


Input is:
- $f$: an associative binary function
- $a$ is the sequence
- $id$ is the **left identity** of $f$ $\:\: \equiv \:\:$ $f(id, x) = x$ for all $x \in \alpha$

Returns:
- a tuple containing:
  - a value of type $S_\alpha$, the sequence of intermediate values
  - a value of type $\alpha$ that is the result of the "sum" with respect to $f$ of the input sequence $a$



<br>


<br>

$scan \: f \: id \: a = (\langle reduce \:\: f \:\: id \:\: a[0 \ldots (i-1)] : 0 \le i < |a| \rangle,$  
$\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: reduce \:\: f \:\: id \:\: a)$

In [6]:
def reduce(f, id_, a):
    # print('a=%s' % a) # for tracing
    if len(a) == 0:
        return id_
    elif len(a) == 1:
        return a[0]
    else:
        # can call these in parallel
        return f(reduce(f, id_, a[:len(a)//2]),
                  reduce(f, id_, a[len(a)//2:]))
        
def plus(x, y):
    return x + y

def scan(f, id_, a):
    """
    This is a horribly inefficient implementation of scan
    only to understand what it does.
    We'll discuss how to make it more efficient later.
    """
    return (
            [reduce(f, id_, a[:i+1]) for i in range(len(a))],
             reduce(f, id_, a)
           )

scan(plus, 0, [2,1,3,2,2,5,4,1])

([2, 3, 6, 8, 10, 15, 19, 20], 20)

`scan` is sometimes called `prefix sum`, as in the previous example it computes the sum of every prefix of a list.

In [5]:
# what does this do?
import math
def gt(x,y):
    return x if x > y else y

scan(gt, -math.inf, [10,4,5,12,3,16,5]) 

([10, 10, 10, 12, 12, 16, 16], 16)